In [1]:
import pandas as pd
import numpy as np
import json
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
import random
warnings.filterwarnings('ignore')
import lightgbm as lgb
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import re
import requests
from itertools import combinations
from tqdm import tqdm_notebook

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1143 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [4]:
# 获取近期所有计划()
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                p.*,
                b.image_id
            FROM
                db_ptom.ptom_third_plan p
            left join
                db_ptom.ptom_plan b
            on p.plan_id=b.plan_id
            WHERE
                p.game_id IN ({})
                AND p.media_id = 16
                AND p.platform = 1
                AND p.create_time>= date( NOW() - INTERVAL 1440 HOUR )
                AND p.create_time<= date(NOW())
                            AND p.plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [5]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1143 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 16 
            AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df

In [6]:
def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 16
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df

In [7]:
# 解析json
def get_plan_json(plan_info):
    plan_info.drop(['inventory_type', 'budget', 'bid_mode'], axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info.dropna(subset=['ad_info'], inplace=True)

    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)

    temp = plan_info['targeting'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('targeting', axis=1, inplace=True)

    temp = plan_info['deep_conversion_spec'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('deep_conversion_spec', axis=1, inplace=True)

    if 'behavior_or_interest' in plan_info.columns:
        temp = plan_info['behavior_or_interest'].apply(pd.Series)
        plan_info = pd.concat([plan_info, temp], axis=1)
        plan_info.drop('behavior_or_interest', axis=1, inplace=True)
        plan_info.drop(0, axis=1, inplace=True)
    
        if 'intention' in plan_info.columns:
            temp = plan_info['intention'].apply(pd.Series)
            plan_info = pd.concat([plan_info, temp], axis=1)
            plan_info.drop('intention', axis=1, inplace=True)
            plan_info = plan_info.rename(columns={'targeting_tags': 'intention_targeting_tags'})
            plan_info.drop(0, axis=1, inplace=True)
        else:
            plan_info['intention_targeting_tags'] = np.nan
        
        if 'interest' in plan_info.columns:
            temp = plan_info['interest'].apply(pd.Series)
            plan_info = pd.concat([plan_info, temp], axis=1)
            plan_info.drop('interest', axis=1, inplace=True)
            plan_info = plan_info.rename(
                columns={'category_id_list': 'interest_category_id_list', 'keyword_list': 'interest_keyword_list',
                         'targeting_tags': 'interest_targeting_tags'})
            plan_info.drop(0, axis=1, inplace=True)
        else:
            plan_info['interest_category_id_list'] = np.nan
            plan_info['interest_keyword_list'] = np.nan
            plan_info['interest_targeting_tags'] = np.nan
        
        if 'behavior' in plan_info.columns:
            temp = plan_info['behavior'].apply(pd.Series)
            plan_info = pd.concat([plan_info, temp], axis=1)
            plan_info.drop('behavior', axis=1, inplace=True)
            temp = plan_info[0].apply(pd.Series)
            plan_info = pd.concat([plan_info, temp], axis=1)
            plan_info.drop(0, axis=1, inplace=True)
            plan_info = plan_info.rename(columns={'category_id_list': 'behavior_category_id_list',
                                              'intensity': 'behavior_intensity',
                                              'keyword_list': 'behavior_keyword_list',
                                              'scene': 'behavior_scene',
                                              'targeting_tags': 'behavior_targeting_tags',
                                              'time_window': 'behavior_time_window'})
        else:
            plan_info['behavior_category_id_list'] = np.nan
            plan_info['behavior_intensity'] = np.nan
            plan_info['behavior_keyword_list'] = np.nan
            plan_info['behavior_scene'] = np.nan
            plan_info['behavior_targeting_tags'] = np.nan
            plan_info['behavior_time_window'] = np.nan
    
    else:
        plan_info['intention_targeting_tags'] = np.nan
        plan_info['interest_category_id_list'] = np.nan
        plan_info['interest_keyword_list'] = np.nan
        plan_info['interest_targeting_tags'] = np.nan
        plan_info['behavior_category_id_list'] = np.nan
        plan_info['behavior_intensity'] = np.nan
        plan_info['behavior_keyword_list'] = np.nan
        plan_info['behavior_scene'] = np.nan
        plan_info['behavior_targeting_tags'] = np.nan
        plan_info['behavior_time_window'] = np.nan
        

    
    if 'excluded_converted_audience' in plan_info.columns:
        temp = plan_info['excluded_converted_audience'].apply(pd.Series)
        plan_info = pd.concat([plan_info, temp], axis=1)
        plan_info.drop('excluded_converted_audience', axis=1, inplace=True)
        plan_info.drop(0, axis=1, inplace=True)
    else:
        plan_info['excluded_dimension'] = np.nan
    
    if 'geo_location' in plan_info.columns:
        temp = plan_info['geo_location'].apply(pd.Series)
        plan_info = pd.concat([plan_info, temp], axis=1)
        plan_info.drop('geo_location', axis=1, inplace=True)
        plan_info.drop(0, axis=1, inplace=True)
    else:
        plan_info['location_types'] = np.nan
        plan_info['regions'] = np.nan


    # 过滤一对多计划
    plan_info['ad_id_count'] = plan_info.groupby('plan_id')['ad_id'].transform('count')
    plan_info = plan_info[plan_info['ad_id_count'] == 1]

    # 删除纯买激活的计划
    plan_info = plan_info[~((plan_info['deep_conversion_type'].isna()) & (
            plan_info['optimization_goal'] == 'OPTIMIZATIONGOAL_APP_ACTIVATE'))]
    # 删除auto_audience=True 的记录，并且删除auto_audience字段
    plan_info[plan_info['auto_audience'] == False]
    for col in ['ad_account_id', 'game_id', 'channel_id', 'source_id', 'budget_mode',
                           'create_time', 'image_id', 'optimization_goal', 'time_series',
                           'bid_strategy', 'bid_amount', 'daily_budget', 'expand_enabled',
                           'expand_targeting', 'device_price', 'app_install_status',
                           'gender', 'game_consumption_level', 'age', 'network_type',
                           'deep_conversion_type', 'deep_conversion_behavior_spec',
                           'deep_conversion_worth_spec', 'intention_targeting_tags',
                           'interest_category_id_list', 'interest_keyword_list',
                           'behavior_category_id_list',
                           'behavior_intensity', 'behavior_keyword_list', 'behavior_scene',
                           'behavior_time_window',
                           'conversion_behavior_list', 'excluded_dimension', 'location_types',
                           'regions']:
        if col in plan_info.columns:
            pass
        else:
            plan_info[col] = np.nan
    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'budget_mode',
                           'create_time', 'image_id', 'optimization_goal', 'time_series',
                           'bid_strategy', 'bid_amount', 'daily_budget', 'expand_enabled',
                           'expand_targeting', 'device_price', 'app_install_status',
                           'gender', 'game_consumption_level', 'age', 'network_type',
                           'deep_conversion_type', 'deep_conversion_behavior_spec',
                           'deep_conversion_worth_spec', 'intention_targeting_tags',
                           'interest_category_id_list', 'interest_keyword_list',
                           'behavior_category_id_list',
                           'behavior_intensity', 'behavior_keyword_list', 'behavior_scene',
                           'behavior_time_window',
                           'conversion_behavior_list', 'excluded_dimension', 'location_types',
                           'regions']]
    return plan_info

In [8]:
# 获取近期30天优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
       SELECT
            b.chl_user_id AS channel_id,
            b.source_id,
            b.image_id,
            a.creative_param
        FROM
            db_ptom.ptom_batch_ad_task a
            LEFT JOIN db_ptom.ptom_plan b ON a.plan_name = b.plan_name 
        WHERE
            a.media_id = 16 
            AND b.create_time >= date( NOW() - INTERVAL 1440 HOUR )    
            AND a.game_id IN ({})
            AND b.image_id is not null

    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result

In [9]:
# 获取score_image (分数大于550的image_id)
# def get_score_image():
#     conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
#                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
#     cursor = conn.cursor()
#     sql_engine = 'set hive.execution.engine=tez'
#     sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id=16 and score>=520 and dt=CURRENT_DATE group by image_id,label_ids'
#     cursor.execute(sql_engine)
#     cursor.execute(sql)
#     result = as_pandas(cursor)
#     result['label_ids'] = result['label_ids'].astype(str)
#     result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
#     result['label_ids'] = pd.to_numeric(result['label_ids'], errors='coerce')

#     # 关闭链接
#     cursor.close()
#     conn.close()

#     return result['image_id'].values

In [9]:
# 获取近3天回款率高的素材
def get_good_image():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.image_id AS 'image_id',
            a.image_name AS 'image_name',
            a.media_id,
            a.label_ids,
            ifnull( a.image_run_date_amount, 0 ) AS 'image_run_date_amount',
            ifnull( a.image_create_role_pay_num, 0 ) AS 'image_create_role_pay_num',
            ifnull( a.image_create_role_num, 0 ) AS 'image_create_role_num',
            ifnull( a.image_create_role_pay_sum, 0 ) AS 'image_create_role_pay_sum',
            ifnull( a.image_source_num, 0 ) AS 'image_source_num',
            (
            CASE

                    WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                    0 ELSE IFNULL( a.image_create_role_pay_num, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                END 
                ) AS 'image_create_role_pay_rate',
                (
                CASE

                        WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                    END 
                    ) AS 'image_create_role_cost',
                    (
                    CASE

                            WHEN ifnull( a.image_create_role_pay_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_pay_num, 0 ) 
                        END 
                        ) AS 'image_create_role_pay_cost',
                        (
                        CASE

                                WHEN ifnull( a.image_run_date_amount, 0 )= 0 THEN
                                0 ELSE IFNULL( a.image_create_role_pay_sum, 0 ) / ifnull( a.image_run_date_amount, 0 ) 
                            END 
                            ) AS 'image_create_role_roi' 
                        FROM
                            (
                            SELECT
                                aa.image_id AS 'image_id',
                                aa.image_name AS 'image_name',
                                aa.media_id AS 'media_id',
                                aa.label_ids AS 'label_ids',
                                sum( aa.amount ) AS 'image_run_date_amount',
                                IFNULL( sum( aa.create_role_num ), 0 ) AS 'image_create_role_num',
                                IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'image_create_role_pay_num',
                                IFNULL( sum( bb.new_role_money ), 0 ) AS 'image_create_role_pay_sum',
                                IFNULL( sum( aa.image_source_num ), 0 ) AS 'image_source_num' 
                            FROM
                                (
                                SELECT
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id,
                                    a.media_id,
                                    c.image_id,
                                    c.image_name,
                                    c.label_ids,
                                    IFNULL( sum( a.amount ), 0 ) AS amount,
                                    IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                                    count( DISTINCT b.plan_id ) AS 'image_source_num' 
                                FROM
                                    db_stdata.st_lauch_report a
                                    INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                    AND a.source_id = b.source_id 
                                    AND a.channel_id = b.chl_user_id
                                    LEFT JOIN db_data_ptom.ptom_image_info c ON b.image_id = c.image_id 
                                WHERE
                                    a.tdate_type = 'day' 
                                    AND a.tdate <= date( NOW()) AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 3 DAY ) 
                                    AND a.amount > 0 
                                    AND a.media_id IN ( 10, 16 ) 
                                    AND b.image_id IS NOT NULL 
                                    AND b.image_id <> '' 
                                    AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1143 AND dev_game_id IS NOT NULL ) 
                                GROUP BY
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id 
                                HAVING
                                    c.image_id IS NOT NULL 
                                    AND c.image_name <> '' 
                                ) aa
                                LEFT JOIN (
                                SELECT
                                    game_id,
                                    channel_id,
                                    source_id,
                                    IFNULL( sum( m.new_role_money ), 0 ) AS new_role_money,
                                    IFNULL( sum( m.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                FROM
                                    (
                                    SELECT
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id,
                                        IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                                        IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_lauch_report a
                                        INNER JOIN db_data_ptom.ptom_plan pp ON ( a.game_id = pp.game_id AND a.channel_id = pp.chl_user_id AND a.source_id = pp.source_id ) 
                                    WHERE
                                        a.tdate = date(
                                        NOW()) 
                                        AND a.tdate_type = 'day' 
                                    GROUP BY
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id 
                                    HAVING
                                        ( new_role_money > 0 OR pay_role_user_num > 0 ) UNION ALL
                                    SELECT
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id,
                                        sum( c.create_role_money ) new_role_money,
                                        IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_game_days c 
                                    WHERE
                                        c.report_days = 3 
                                        AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                                        AND c.tdate_type = 'day' 
                                        AND c.query_type = 13 
                                    GROUP BY
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id 
                                    HAVING
                                    ( new_role_money > 0 OR pay_role_user_num > 0 )) m 
                                GROUP BY
                                    game_id,
                                    channel_id,
                                    source_id 
                                ) bb ON aa.game_id = bb.game_id 
                                AND aa.channel_id = bb.channel_id 
                                AND aa.source_id = bb.source_id 
                            GROUP BY
                                aa.image_id,
                                aa.image_name,
                                aa.media_id 
                            HAVING
                            image_run_date_amount >= 100 
            ) a
            WHERE a.image_create_role_pay_sum >0 and 
                a.media_id = 16
                ORDER BY image_create_role_roi Desc
    '''
#     finalSql = sql.format(game_id)
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    result_df = result_df[['image_id','image_run_date_amount','image_create_role_pay_num','image_create_role_pay_sum',
                        'image_create_role_pay_cost','image_create_role_roi']]
    result_df['roi_rank'] = result_df['image_create_role_roi'].rank(ascending=False)
    result_df = result_df[result_df['roi_rank']<=np.ceil((result_df.shape[0] / 2))]
    return result_df['image_id'].values

In [10]:
# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_data_ptom.ptom_plan a 
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 16 
            AND b.game_id IN ({}) 
            AND b.amount >= 200  
            AND b.pay_role_user_num >= 2 
            AND b.new_role_money >= 20
            AND (b.new_role_money / b.amount)>=0.02
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [11]:
plan_info = get_plan_info()
plan_info = get_plan_json(plan_info)
creative_info = get_creative()
image_info = get_image_info()
launch_report = get_launch_report()
creative_info['creative_param'] = creative_info['creative_param'].apply(json.loads)
temp = creative_info['creative_param'].apply(pd.Series)
if 'image_id' in temp.columns:
    temp = temp.drop('image_id', axis=1)
creative_info = pd.concat([creative_info, temp], axis=1)
creative_info.drop('creative_param', axis=1, inplace=True)
creative_info.drop(['title', 'adcreative_template_parent', 'idea_type', 'adcreative_name', 'ideaName', '_creative'],
                   axis=1, inplace=True)
creative_info['adcreative_elements_array'] = creative_info['adcreative_elements_array'].apply(lambda x: x[0])
creative_info['adcreative_elements'] = creative_info['adcreative_elements_array'].apply(
    lambda x: x['adcreative_elements'])
creative_info.drop('adcreative_elements_array', axis=1, inplace=True)
for col in ['channel_id', 'source_id', 'image_id', 'deep_link_url',
               'adcreative_template_id', 'page_spec', 'page_type', 'site_set', 'label',
               'promoted_object_id', 'promoted_object_type', 'automatic_site_enabled',
               'link_name_type', 'link_page_type', 'profile_id', 'link_page_spec',
               'adcreative_elements']:
    if col in creative_info.columns:
        pass
    else:
        creative_info[col] = np.nan

creative_info = creative_info[['channel_id', 'source_id', 'image_id', 'deep_link_url',
                               'adcreative_template_id', 'page_spec', 'page_type', 'site_set', 'label',
                               'promoted_object_id', 'promoted_object_type', 'automatic_site_enabled',
                               'link_name_type', 'link_page_type', 'profile_id', 'link_page_spec',
                               'adcreative_elements']]
plan_info = pd.merge(plan_info.drop(['image_id'], axis=1), creative_info, on=['channel_id', 'source_id'],
                     how='inner')
plan_info.dropna(subset=['image_id'], inplace=True)
plan_info['image_id'] = plan_info['image_id'].astype(int)
plan_info['adcreative_template_id'] = plan_info['adcreative_template_id'].astype(int)

plan_info['profile_id'] = plan_info['profile_id'].apply(lambda x: x if x != x else str(int(x)))
plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
plan_info_current = plan_info[plan_info['create_time'] >= pd.datetime.now() - pd.DateOffset(30)]  ## TODO:时间线拉长
now_plan_roi = get_now_plan_roi()
# score_image = get_score_image()
image_info = image_info[image_info['image_id'].notna()]
image_info['image_id'] = image_info['image_id'].astype(int)


In [20]:
df_create = pd.merge(plan_info_current, image_info, on=['channel_id', 'source_id', 'image_id'], how='left')
df_create = pd.merge(df_create, now_plan_roi.drop(['ad_account_id'], axis=1), on=['channel_id', 'source_id'],
                     how='inner')
# df_create = df_create[df_create['site_set'].notna()]  ## 注释掉：以支持自动化版位

In [21]:
df_create.to_csv('./df_create.csv',index=0)

In [22]:
good_image = get_good_image()

In [23]:
good_image

array([40601, 40827, 41562, 41335, 41095, 41336, 41222, 41060, 41218,
       41535, 40619, 41627, 41572, 41617, 41402, 41581, 41496, 41290,
       40622, 41382, 41577, 41514], dtype=int64)

In [24]:
# df_create['site_set'] = df_create['site_set'].map(str)
# df_create['site_set'] = df_create['site_set'].replace('nan',"[]")

In [28]:
# df_create['site_set'] = df_create['site_set'].apply(ast.literal_eval)

In [32]:
game_id = 1001854  ## 选择包：n1计划-IOS联运-IOS联运
# df_create = df_create[df_create['game_id'].isin([1001617, 1001447, 1001446, 1001611, 1001613, 1001444, 1001609, 1001610])]
## 候选计划的包ID限制（明日战纪-ios-IOS联运，n1计划-IOS联运-IOS联运，幸存挑战 - 曙光信仰-IOS联运, 迷岛生存IOS-IOS联运, 潜藏之地-IOS联运,
# 绝地末日 - IOS联运 - IOS联运# , 迷城起源-黎明危域-IOS联运, 黎明血战-IOS-IOS联运）
# image_id_group = np.intersect1d(df_create['image_id'].unique(), score_image)
# image_id_group = df_create['image_id'].unique()
image_id_group = np.intersect1d(df_create['image_id'].unique(), good_image)
print(image_id_group)

[40601 40619 40827 41060 41218 41222 41402 41581 41617 41627]


In [33]:
image_adcreative_elements = df_create[['image_id','adcreative_template_id']]
image_adcreative_elements = image_adcreative_elements[image_adcreative_elements['image_id'].isin(image_id_group)]
image_adcreative_elements = image_adcreative_elements.drop_duplicates(subset='image_id', keep='first')

In [ ]:
## 跑自动版位
# df_create['site_set'] = df_create['site_set'].map(str)
df_create_s = df_create[(df_create['site_set'].isna()) & ((df_create['adcreative_template_id'] == 720)|(df_create['adcreative_template_id'] == 721))]  ## TODO

In [35]:
def create_plan(df, image_adcreative_elements):
    # 选ad_account_id、image_id每个账号+素材8条
    game_id = 1001854  ## 选择包：

    plan = image_adcreative_elements.copy()
    plan = plan.reset_index(drop=True)

    plan['game_id'] = game_id
    
    # 选择预算，不限制预算
    plan['budget_mode'] = 0
    plan['daily_budget'] = 0
    
    sample_df = df[['expand_enabled', 'expand_targeting','device_price', 'app_install_status', 'gender', 'game_consumption_level', 
        'age', 'network_type', 'conversion_behavior_list', 'excluded_dimension', 'location_types', 'regions',
        'intention_targeting_tags', 'interest_category_id_list', 'interest_keyword_list',
        'behavior_category_id_list', 'behavior_intensity', 'behavior_keyword_list', 'behavior_scene', 'behavior_time_window',
        'site_set', 'deep_link_url', 'page_spec', 'page_type', 'link_page_spec','adcreative_elements',
        'link_name_type', 'link_page_type', 'promoted_object_id', 'profile_id', 'promoted_object_type',
        'automatic_site_enabled', 'label',
        'optimization_goal', 'bid_strategy', 'bid_amount', 'deep_conversion_type', 'deep_conversion_behavior_spec',
        'deep_conversion_worth_spec', 'time_series']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    
    plan = pd.concat([plan, sample_df], axis=1)

#     plan['site_set'] = plan['site_set'].apply(lambda x:ast.literal_eval(x) if x != "nan" and not pd.isnull(x) else [])  ## TODO:支持自动化版位修改

    plan['promoted_object_id'] = '2000002794'    ## TODO  七雄
#     plan['create_time'] = pd.to_datetime(pd.datetime.now())
#     plan['create_date'] = pd.to_datetime(pd.datetime.now().date())
    plan = plan.reset_index(drop=True)
    
    return plan

In [36]:
plan_create = create_plan(df_create_s, image_adcreative_elements)

In [37]:
plan_create.shape

(10, 45)

In [62]:
ad_account_id_group = np.array([9561, 9562, 9563, 9564, 9565])
# num_pre = int(plan_create.shape[0] / len(ad_account_id_group))
plan_num = 1

In [63]:
plan_result = pd.DataFrame()
for ad_account_id in ad_account_id_group:
    plan_result_ = plan_create.sample(n=plan_num, replace=False).reset_index(drop=True)
#     plan_create = plan_create.drop(plan_create[plan_create['image_id'].isin(plan_result_.image_id.values)].index, axis=0)
    plan_result_['ad_account_id'] = ad_account_id
    plan_result = plan_result.append(plan_result_)

In [64]:
plan_result.shape

(5, 46)

In [67]:
plan_result

,image_id,adcreative_template_id,game_id,budget_mode,daily_budget,expand_enabled,expand_targeting,device_price,app_install_status,gender,game_consumption_level,age,network_type,conversion_behavior_list,excluded_dimension,location_types,regions,intention_targeting_tags,interest_category_id_list,interest_keyword_list,behavior_category_id_list,behavior_intensity,behavior_keyword_list,behavior_scene,behavior_time_window,site_set,deep_link_url,page_spec,page_type,link_page_spec,adcreative_elements,link_name_type,link_page_type,promoted_object_id,profile_id,promoted_object_type,automatic_site_enabled,label,optimization_goal,bid_strategy,bid_amount,deep_conversion_type,deep_conversion_behavior_spec,deep_conversion_worth_spec,time_series,ad_account_id
0,41402,721,1001854,0,0,True,"[age, gender]",NaN,[NOT_INSTALLED],[MALE],"[HIGH, NORMAL]","[{'min': 21, 'max': 48}]",NaN,[OPTIMIZATIONGOAL_APP_ACTIVATE],EXCLUDED_DIMENSION_APP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SITE_SET_QQ_MUSIC_GAME', 'SITE_SET_KANDIAN',...",,{'page_id': '2314272060'},PAGE_TYPE_TSA_APP,NaN,"{'image': 127996, 'head_line': '七雄页游原班打造，正版授权！...",NaN,NaN,2000002794,NaN,PROMOTED_OBJECT_TYPE_APP_ANDROID,False,NaN,OPTIMIZATIONGOAL_APP_ACTIVATE,BID_STRATEGY_TARGET_COST,68252,DEEP_CONVERSION_WORTH,NaN,"{'goal': 'GOAL_1DAY_PURCHASE_ROAS', 'expected_...",1111111111111111111111111111111111111111111111...,9561
1,41617,720,1001854,0,0,True,"[age, gender]",NaN,NaN,NaN,NaN,"[{'min': 21, 'max': 48}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SITE_SET_QQ_MUSIC_GAME', 'SITE_SET_KANDIAN',...",,{'page_id': '2314272060'},PAGE_TYPE_TSA_APP,NaN,"{'image': 128225, 'head_line': '七雄页游原班打造，正版授权！...",NaN,NaN,2000002794,NaN,PROMOTED_OBJECT_TYPE_APP_ANDROID,False,NaN,OPTIMIZATIONGOAL_APP_ACTIVATE,BID_STRATEGY_TARGET_COST,84816,DEEP_CONVERSION_WORTH,NaN,"{'goal': 'GOAL_1DAY_PURCHASE_ROAS', 'expected_...",1111111111111111111111111111111111111111111111...,9562
2,40827,720,1001854,0,0,True,"[age, gender]",NaN,NaN,NaN,NaN,"[{'min': 21, 'max': 48}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SITE_SET_QQ_MUSIC_GAME', 'SITE_SET_KANDIAN',...",,{'page_id': '2314272060'},PAGE_TYPE_TSA_APP,NaN,"{'image': 127591, 'head_line': '开局80万大军，你能一统七国...",NaN,NaN,2000002794,NaN,PROMOTED_OBJECT_TYPE_APP_ANDROID,False,NaN,OPTIMIZATIONGOAL_APP_ACTIVATE,BID_STRATEGY_TARGET_COST,47856,DEEP_CONVERSION_WORTH,NaN,"{'goal': 'GOAL_1DAY_PURCHASE_ROAS', 'expected_...",1111111111111111111111111111111111111111111111...,9563
3,40601,720,1001854,0,0,True,"[age, gender]",NaN,[NOT_INSTALLED],[MALE],"[HIGH, NORMAL]","[{'min': 21, 'max': 48}]",NaN,[OPTIMIZATIONGOAL_APP_ACTIVATE],EXCLUDED_DIMENSION_APP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SITE_SET_QQ_MUSIC_GAME', 'SITE_SET_KANDIAN',...",,{'page_id': '2314272060'},PAGE_TYPE_TSA_APP,NaN,"{'image': 127996, 'head_line': '七雄页游原班打造，正版授权！...",NaN,NaN,2000002794,NaN,PROMOTED_OBJECT_TYPE_APP_ANDROID,False,NaN,OPTIMIZATIONGOAL_APP_ACTIVATE,BID_STRATEGY_TARGET_COST,68252,DEEP_CONVERSION_WORTH,NaN,"{'goal': 'GOAL_1DAY_PURCHASE_ROAS', 'expected_...",1111111111111111111111111111111111111111111111...,9564
4,41222,720,1001854,0,0,True,"[age, gender]",NaN,NaN,NaN,NaN,"[{'min': 21, 'max': 48}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SITE_SET_QQ_MUSIC_GAME', 'SITE_SET_KANDIAN',...",,{'page_id': '2314272060'},PAGE_TYPE_TSA_APP,NaN,"{'image': 127591, 'head_line': '开局80万大军，你能一统七国...",NaN,NaN,2000002794,NaN,PROMOTED_OBJECT_TYPE_APP_ANDROID,False,NaN,OPTIMIZATIONGOAL_APP_ACTIVATE,BID_STRATEGY_TARGET_COST,47856,DEEP_CONVERSION_WORTH,NaN,"{'goal': 'GOAL_1DAY_PURCHASE_ROAS', 'expected_...",1111111111111111111111111111111111111111111111...,9565


In [66]:
# [SITE_SET_WECHAT] 公众号和小程序adcreative_template_id只跑1480、560、720、721、1064五种
# plan_result['site_set'] = plan_result['site_set'].map(str)
# plan_result['site_set'] = "['SITE_SET_MOMENTS']"  ## TODO 固定跑朋友圈
plan_result = plan_result[~((plan_result['site_set'] == "['SITE_SET_WECHAT']") & (
    ~plan_result['adcreative_template_id'].isin([1480, 560, 720, 721, 1064])))]

plan_result['location_types'] = plan_result['location_types'].apply(lambda x: ['LIVE_IN'] if x == x else x)

## TODO 暂时不跑朋友圈和公众号
# # 修改落地页ID   N1 计划
plan_result['page_spec'] = plan_result.apply(
    lambda x: {'override_canvas_head_option': 'OPTION_CREATIVE_OVERRIDE_CANVAS',
               'page_id': '2314299445'} if x.site_set == "['SITE_SET_MOMENTS']"
    else ({'override_canvas_head_option': 'OPTION_CREATIVE_OVERRIDE_CANVAS',
           'page_id': '2314272060'} if x.site_set == "['SITE_SET_WECHAT']" else np.nan), axis=1)

# plan_result['page_spec'] = np.where(plan_result.site_set.str.contains('SITE_SET_WECHAT') & plan_result.site_set != "['SITE_SET_WECHAT']", {'page_id': '2314272060'}, plan_result.page_spec)
# 朋友圈头像ID
plan_result['ad_account_id'] = plan_result['ad_account_id'].map(str)
plan_result_1 = plan_result[plan_result['site_set'] == "['SITE_SET_MOMENTS']"]
plan_result_2 = plan_result[plan_result['site_set'] != "['SITE_SET_MOMENTS']"]

# 七雄
profile_id_dict = {'9561': '663136', '9562': '663143', '9563': '663157', '9564': '663169', '9565': '663178'}

plan_result_1['profile_id'] = plan_result_1['ad_account_id'].map(profile_id_dict)

plan_result = plan_result_1.append(plan_result_2)
plan_result = plan_result.reset_index(drop=True)

# 年龄定向
plan_result['age'] = plan_result['age'].apply(lambda x: [{'min': 21, 'max': 48}])

In [160]:
## 721是竖版，720是横版
def image_size(image_id, size):
    """ 匹配：素材-版位 """
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    sql = '''
            /*手动查询*/ 
            SELECT
                id,
                image_id,
                image_name,
                width,
                height,
                size
            FROM
                db_ptom.ptom_image_detail_info
            WHERE
                image_id = {}
        '''
    new_ids = pd.read_sql(sql.format(image_id), conn)  ## 新素材-主ID
#     size = (1280,720)
    # 匹配
    new_id = new_ids[(new_ids['width']==size[0]) & (new_ids['height']==size[1])]['id']
    assert len(new_id) > 0, "主素材{}找不到对应的{}尺寸".format(image_id, (size[0], size[1]))
    return new_id.values[0]

In [161]:
for i in range(plan_result.shape[0]):
    a = plan_result.loc[i,'adcreative_elements'].copy()
    if plan_result.loc[i,'adcreative_template_id'] == 720:
        image_detail_id = image_size(plan_result.loc[i,'image_id'],(1280,720))

        a['video'] = image_detail_id
        a['image'] = image_detail_id
        plan_result.loc[i,'adcreative_elements'] = str(a)
    elif plan_result.loc[i,'adcreative_template_id'] == 721:
        image_detail_id = image_size(plan_result.loc[i,'image_id'],(720,1280))
        a['video'] = image_detail_id
        a['image'] = image_detail_id
        plan_result.loc[i,'adcreative_elements'] = str(a)

In [162]:
# 定义组合json
plan_result['intention'] = plan_result['intention_targeting_tags'].apply(lambda x: {'targeting_tags': x})
plan_result.drop(['intention_targeting_tags'], axis=1, inplace=True)

ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(
        json.loads(plan_result.loc[i, ['interest_category_id_list', 'interest_keyword_list']].rename(index={
            'interest_category_id_list': 'category_id_list',
            'interest_keyword_list': 'keyword_list'}).to_json()))
plan_result['interest'] = ad_info
plan_result.drop(['interest_category_id_list', 'interest_keyword_list'], axis=1, inplace=True)

ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, 'behavior_category_id_list':'behavior_time_window'].rename(index={
        'behavior_category_id_list': 'category_id_list',
        'behavior_intensity': 'intensity',
        'behavior_keyword_list': 'keyword_list',
        'behavior_scene': 'scene',
        'behavior_time_window': 'time_window'}).to_json()))
plan_result['behavior'] = ad_info
plan_result.drop(['behavior_category_id_list', 'behavior_intensity', 'behavior_keyword_list', 'behavior_scene',
                  'behavior_time_window'], axis=1, inplace=True)

plan_result['behavior'] = plan_result['behavior'].apply(lambda x: [x])

ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, ['conversion_behavior_list', 'excluded_dimension']].to_json()))
plan_result['excluded_converted_audience'] = ad_info
plan_result.drop(['conversion_behavior_list', 'excluded_dimension'], axis=1, inplace=True)

ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, ['regions', 'location_types']].to_json()))
plan_result['geo_location'] = ad_info
plan_result.drop(['regions', 'location_types'], axis=1, inplace=True)

ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[
                                  i, ["device_price", "app_install_status", "gender", "game_consumption_level",
                                      "age", "network_type",
                                      "excluded_converted_audience", "geo_location",
                                      "intention", "interest", "behavior"]].to_json()))
plan_result['targeting'] = ad_info
plan_result.drop(["device_price", "app_install_status", "gender", "game_consumption_level", "age", "network_type",
                  "excluded_converted_audience", "geo_location",
                  "intention", "interest", "behavior"], axis=1, inplace=True)


In [163]:
# plan_result['operation'] = 'disable'
plan_result['plan_auto_task_id'] = "11427,12114"
plan_result['op_id'] = 13268
plan_result['flag'] = 'GDT'
plan_result['game_name'] = '七雄'
plan_result['platform'] = 1
plan_result['ad_account_id'] = plan_result['ad_account_id'].astype(int)
# plan_result['site_set'] = plan_result['site_set'].apply(ast.literal_eval)
# plan_result['adcreative_template_id'] = 720  # XQXS横版素材

# plan_result['link_name_type'] = 'DOWNLOAD_APP'  # 下载app
# # 一键起量
# plan_result['auto_acquisition_enabled'] = True
# plan_result['auto_acquisition_budget'] = 11111
# # 开启自动扩量
# plan_result['expand_enabled'] = True
# plan_result['expand_targeting'] = plan_result['expand_targeting'].apply(lambda x: [])
# # # 开启加速投放
# plan_result['speed_mode'] = 'SPEED_MODE_FAST'

# 固定品牌名
plan_result = plan_result.reset_index(drop=True)
plan_result['adcreative_elements'] = plan_result['adcreative_elements'].apply(ast.literal_eval)
for i in range(plan_result.shape[0]):
    a = plan_result.loc[i, 'adcreative_elements'].copy()
    a['brand'] = {'brand_name': '七雄争霸', 'brand_img': 123609}
    plan_result.loc[i, 'adcreative_elements'] = str(a)

plan_result['adcreative_elements'] = plan_result['adcreative_elements'].apply(ast.literal_eval)

# 出价方式  ##TODO  固定ROI出价
plan_result['optimization_goal'] = plan_result['optimization_goal'].apply(lambda x: 'OPTIMIZATIONGOAL_APP_ACTIVATE')
plan_result['deep_conversion_type'] = plan_result['optimization_goal'].apply(
    lambda x: 'DEEP_CONVERSION_WORTH' if x == 'OPTIMIZATIONGOAL_APP_ACTIVATE' else np.nan)
plan_result['deep_conversion_worth_spec'] = plan_result['optimization_goal'].apply(
    lambda x: {'goal': 'GOAL_1DAY_PURCHASE_ROAS',
               'expected_roi': 0.036} if x == 'OPTIMIZATIONGOAL_APP_ACTIVATE' else np.nan)
plan_result['deep_conversion_behavior_spec'] = np.nan
# 固定出价
plan_result['bid_amount'] = plan_result['optimization_goal'].apply(
    lambda x: random.randint(250000, 280000) if x == 'OPTIMIZATIONGOAL_APP_ACTIVATE'
    else (random.randint(210000, 220000) if x == 'OPTIMIZATIONGOAL_APP_PURCHASE'
          else (random.randint(430000, 440000))))

# 周三周四更新，凌晨不跑计划
# plan_result['time_series'] = plan_result['time_series'].apply(
#     lambda x: x[0:96] + '1111111111000000000011' + x[118:144] + '1111111111000000000011' + x[166:])
# plan_result['time_series'] = plan_result['time_series'].apply(lambda x:"111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100000000001111111111111111111111111111111111111100000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111")
plan_result.drop('page_type', axis=1, inplace=True)
plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志

In [164]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
    open_api_url_prefix = "https://ptom.caohua.com/"
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"   ## 预发布环境
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 16
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [165]:
print(plan_result.shape[0])
plan_result_seg = pd.DataFrame()
for i in range(plan_result.shape[0]):
    plan_result_seg = plan_result_seg.append(plan_result.iloc[i:i + 1])
    if (i > 0 and i % 40 == 0) or i == (plan_result.shape[0] - 1):
        plan_result_seg = plan_result_seg.reset_index(drop=True)
        print(plan_result_seg.shape[0])
        rsp_data = get_ad_create(plan_result_seg)
        print(rsp_data)
        time.sleep(200)
        plan_result_seg = pd.DataFrame()

30
30
结束....
{'code': 200, 'data': True, 'msg': ''}


In [7]:

def foo():
    print("starting...")
    while True:
        res = yield 4
        print("res:",res)



In [8]:
g = foo()

In [9]:
print(next(g))

starting...
4


In [10]:
print(next(g))

res: None
4


In [56]:
df = pd.DataFrame({'A':list('abcd')})

In [59]:
df

,A,B
0,a,1
1,b,b
2,c,c
3,d,d


In [58]:
df['B'] = np.where(df['A'].str.contains('a'), 1, df.A)

In [48]:
df[df.A.str.contains('a')]

,A
0,a
